## Book search

In [1]:
import pandas as pd

df = pd.read_csv("Goodreads_books_with_genres.csv")
df['publication_date'] = pd.to_datetime(df['publication_date'], format='mixed', errors= 'coerce') ##converts to date format
df['year'] = df['publication_date'].dt.year ##takes only year from the date as a float

def get_user_input():  
    title = input("Book title: ").strip().lower()  
    author = input("Author name: ").strip().lower()  
    genre = input("Genre: ").strip().lower()  
    min_rating = input("Minimum rating (0-5): ") or 0  ##ratings are from 0 to 5
    max_rating = input("Maximum rating (0-5): ") or 5
    max_pages = input("Maximum pages: ") or None
    date_published = input("Year published: ") or None ##year or no entry, float applied in function
    message = "Didn't find what you were looking for? Try suggestions below!"
    
    filtered = df
    try:
        filtered = df[
        (df['average_rating'] >= float(min_rating)) &  ##searches and limits average_rating to min and max rating entered 
        (df['average_rating'] <= float(max_rating))]
    except ValueError:
        print("WARNING! Incorrect number format for ratings. Please enter a numeric value.")
    try:
        if max_pages is not None:
            filtered = filtered[filtered['num_pages'] <= float(max_pages)]
        else: max_pages = str("")  ##if no number is entered it does not return none but rather empty string
    except ValueError: ##this handles error where user inputs number in non-numerical form, e.g. ten instead of 10
        print("WARNING! Incorrect number format for page number. Please enter a numeric value.")
    
    try:    
        if date_published is not None:  
            filtered = filtered[filtered['year'] == float(date_published)]
        else: date_published = str("") 
    except ValueError: ##handles error with year format not being a number
        print("WARNING! Incorrect number format for year. Please enter a numeric value.")
    
    if title:
        filtered = filtered[filtered['Title'].str.lower().str.contains(title)] ##this finds and filters column title with entered title of book in lowercase
        
    if author:  
        filtered = filtered[filtered['Author'].str.lower().str.contains(author)] 
        
    if genre:  
        filtered = filtered[filtered['genres'].str.lower().str.contains(r'\b' + genre + r'\b', regex=True, na=False)]
    
    if filtered.empty:  ##this reports back when the table is empty, which can be for several reasons
        print("No books found with your search criteria.\n"    
        "Try checking your spelling or adjust your filters.\n")
        
    print(f"Showing results for: \n\nBook title: {title} \nAuthor name: {author} \nGenre: {genre}" 
          f"\nBook rating range (min-max): {min_rating} - {max_rating} \nMaximum page count: {max_pages}" 
          f"\nYear published: {date_published} \n\n{message}")
    return filtered[['Title', 'Author', 'genres', 'average_rating', 'num_pages', 'publication_date']] ##prints the table

In [4]:
test_input = get_user_input()
test_input

Showing results for: 

Book title:  
Author name: agatha christie 
Genre: 
Book rating range (min-max): 0 - 5 
Maximum page count: 
Year published:  

Didn't find what you were looking for? Try suggestions below!


,Title,Author,genres,average_rating,num_pages,publication_date
4517,Cards on the Table (Hercule Poirot #15),Agatha Christie,"Mystery;Fiction;Mystery,Crime;Classics;Thrille...",3.92,324,2005-07-05
4518,A Murder Is Announced (Miss Marple #5),Agatha Christie,"Mystery;Fiction;Mystery,Crime;Classics;Thrille...",3.98,288,2006-09-30
4519,And Then There Were None,Agatha Christie,"Mystery;Classics;Fiction;Mystery,Crime;Thrille...",4.26,264,2004-05-03
4520,Sleeping Murder (Miss Marple #13),Agatha Christie,"Mystery;Fiction;Mystery,Crime;Classics;Thrille...",3.95,242,2003-08-04
4521,Agatha Christie: An Autobiography,Agatha Christie/Robert Welch Herrick,"Nonfiction;Biography;Biography,Autobiography;A...",4.27,560,2001-01-02
4522,Murder on the Orient Express (Hercule Poirot ...,Agatha Christie,"Mystery;Classics;Fiction;Mystery,Crime;Thrille...",4.17,322,2004-08-31
4523,Evil Under the Sun (Hercule Poirot #24),Agatha Christie,"Mystery;Fiction;Mystery,Crime;Classics;Thrille...",3.97,220,2006-09-01
4524,Hallowe'en Party (Hercule Poirot #39),Agatha Christie,"Mystery;Fiction;Mystery,Crime;Classics;Thrille...",3.66,336,2001-09-03
4525,One Two Buckle My Shoe (Hercule Poirot #23),Agatha Christie/Hugh Fraser,"Mystery;Fiction;Mystery,Crime;Classics;Mystery...",3.76,0,2004-03-16
4526,A Caribbean Mystery,Agatha Christie/Rosalind Ayres,"Mystery;Fiction;Mystery,Crime;Classics;Thrille...",3.80,0,2006-10-09


## Book suggestions

#### Top 10 best rated books by genre

In [12]:
def input_suggestions():
    genre = input("Genre: ").strip().lower() 
    filtered = df.sort_values(by= 'average_rating', ascending = False) ##sorts based on the book rating
    if genre:  
        filtered = filtered[filtered['genres'].str.lower().str.contains(r'\b' + genre + r'\b', regex=True, na=False)]
    print(f"Showing results for: {genre}") 
    if filtered.empty:  ##this reports back when the table is empty - genre is mispelled or does not exist in the list
        print("No books found in your genre. Check your spelling or try different genre.\n")
    return filtered[['Title', 'Author', 'genres', 'average_rating','num_pages', 'publication_date']]

In [13]:
test_suggestion = input_suggestions()
test_suggestion.head(10) ##gives 10 top rows from the table

Showing results for: detective


,Title,Author,genres,average_rating,num_pages,publication_date
1056,The New Annotated Sherlock Holmes: The Complet...,Arthur Conan Doyle/Leslie S. Klinger,Mystery;Classics;Fiction;Short Stories;Mystery...,4.64,1878,2004-11-30
1057,The New Annotated Sherlock Holmes: The Novels,Arthur Conan Doyle/Leslie S. Klinger,"Classics;Mystery;Fiction;Mystery,Crime;Mystery...",4.53,907,2005-11-07
2454,Sherlock Holmes and the Case of the Hound of t...,Malvina G. Vogel/Arthur Conan Doyle,"Mystery;Classics;Fiction;Mystery,Crime;Mystery...",4.51,237,2005-01-01
627,Stories and Early Novels: Pulp Stories / The B...,Raymond Chandler/Frank MacShane,"Fiction;Mystery;Short Stories;Mystery,Crime;My...",4.50,1199,1995-10-01
2750,The Complete Sherlock Holmes Volume I,Arthur Conan Doyle/Kyle Freeman,Classics;Mystery;Fiction;Short Stories;Mystery...,4.49,709,2003-09-01
8455,The Harry Bosch Novels Volume 2: The Last Coy...,Michael Connelly,"Mystery;Fiction;Mystery,Crime;Thriller;Mystery...",4.49,821,2003-11-03
2751,The Complete Sherlock Holmes Volume II,Arthur Conan Doyle/Kyle Freeman,Classics;Mystery;Fiction;Short Stories;Mystery...,4.47,709,2003-10-01
1055,Sherlock Holmes: The Complete Novels and Stori...,Arthur Conan Doyle,Classics;Mystery;Fiction;Short Stories;Mystery...,4.47,1059,1986-08-26
626,Later Novels and Other Writings: The Lady in t...,Raymond Chandler/Frank MacShane,"Mystery;Fiction;Mystery,Noir;Mystery,Crime;Mys...",4.47,1076,1995-10-01
628,The Lady in the Lake The Little Sister The L...,Raymond Chandler/Tom Hiney,"Mystery;Mystery,Crime;Fiction;Mystery,Noir;Cla...",4.45,1016,2002-10-15


#### Try your luck! (Random book)

In [11]:
def random_book():
    random_sugg = pd.DataFrame.sample(df, n=1) ## .sample take 1 random row from the whole dataframe
    return random_sugg [['Title', 'Author', 'genres', 'average_rating','num_pages', 'publication_date']]

In [7]:
test_random = random_book()
test_random

,Title,Author,genres,average_rating,num_pages,publication_date
4674,The Diagnosis,Alan Lightman,Fiction;Novels;Literature;Contemporary;Unfinis...,2.86,384,2002-02-19


#### Try your luck, get 10 random books, but choose your preferred genre

In [9]:
def random_books_genre():
    genre = input("Genre: ").strip().lower() 
    filtered = df
    try:
        if genre:  
            filtered = filtered[filtered['genres'].str.lower().str.contains(r'\b' + genre + r'\b', regex=True, na=False)] 
            filtered_genres = pd.DataFrame.sample(filtered, n=10) ## .sample takes a random 10 rows from the filtered books by genre
            print(f"Showing results for: {genre}")     
        return filtered_genres [['Title', 'Author', 'genres', 'average_rating','num_pages', 'publication_date']]
    except ValueError: ##used try-except here because if mispelled genre is entered it returns error because .sample expects 10 of something
          print("No books found in your genre. Please check spelling of the genre entered.")

In [10]:
test_almost_random = random_books_genre()
test_almost_random

Showing results for: detective


,Title,Author,genres,average_rating,num_pages,publication_date
736,The Bone Collector (Lincoln Rhyme #1),Jeffery Deaver,"Mystery;Thriller;Fiction;Mystery,Crime;Thrille...",4.19,528,1998-04-01
9400,Broken Prey (Lucas Davenport #16),John Sandford,"Mystery;Fiction;Thriller;Mystery,Crime;Thrille...",4.18,481,2006-05-02
5861,Double Dealer (CSI: Crime Scene Investigation ...,Max Allan Collins/Mike Flaherty,"Mystery;Mystery,Crime;Fiction;Thriller;Thrille...",3.67,320,2003-04-01
1957,In the Company of Cheerful Ladies (No. 1 Ladie...,Alexander McCall Smith,"Mystery;Fiction;Cultural,Africa;Southern Afric...",4.08,256,2006-03-14
4528,The Body in the Library (Miss Marple #3),Agatha Christie,"Mystery;Fiction;Mystery,Crime;Classics;Thrille...",3.85,191,2006-09-01
8733,Roses Are Red (Alex Cross #6),James Patterson,"Mystery;Fiction;Thriller;Mystery,Crime;Suspens...",4.04,400,2000-11-20
1057,The New Annotated Sherlock Holmes: The Novels,Arthur Conan Doyle/Leslie S. Klinger,"Classics;Mystery;Fiction;Mystery,Crime;Mystery...",4.53,907,2005-11-07
5760,School Days (Spenser #33),Robert B. Parker/Joe Mantegna,"Mystery;Fiction;Mystery,Detective;Mystery,Crim...",3.87,0,2005-09-27
4218,The Mystery of the Whispering Mummy (Alfred Hi...,Robert Arthur/Alfred Hitchcock,Mystery;Fiction;Young Adult;Childrens;Adventur...,3.84,185,1985-09-12
4520,Sleeping Murder (Miss Marple #13),Agatha Christie,"Mystery;Fiction;Mystery,Crime;Classics;Thrille...",3.95,242,2003-08-04
